In [37]:
import robotic as ry
import numpy as np
import random
import time
from bridgebuild_env import create_n_boxes
# this import the local manipulation.py .. to be integrated in robotic..
import manipulation as manip

### Task Env: Bridge Builder

In [38]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
n_boxes = 3
keep_dist_between = ["box1", "box2", "box3"]
C = create_n_boxes(C = C, N = n_boxes, position = 'fixed')

# remove so collision with wrist camera is ignored
C.delFrame("panda_collCameraWrist")

# for convenience, a few definitions:
qHome = C.getJointState()
gripper = "l_gripper";
palm = "l_palm";
box1 = "box1";
box2 = "box2";
box3 = "box3";
table = "table";
boxSize = C.frame(box1).getSize()

C.view()

0

In [39]:
C.setJointState(qHome)
C.view_raise()
boxes_pos = [C.frame(box1).getPosition(), C.frame(box2).getPosition(), C.frame(box3).getPosition()]
boxes = [box1, box2, box3]
last_box_memory = []
for l in range(3):
        box = np.random.choice(boxes)
        qStart = C.getJointState()
        graspDirection = 'x' #random.choice(['y', 'z']) #'x' not possible: box too large
        placeDirection = 'z' #random.choice(['x', 'y', 'z', 'xNeg', 'yNeg', 'zNeg'])
        info = f'placement {l}: grasp {graspDirection} place {placeDirection}'
        print('===', info)

        M = manip.ManipulationModelling(C, info, helpers=[gripper])
        M.setup_pick_and_place_waypoints(gripper, box, homing_scale=1e-1, accumulated_collisions=True)
        M.grasp_box(1., gripper, box, palm, graspDirection)
        print('place direction:', placeDirection)

        if l > 1:

            offset_xyz = [0.0, 0.0, 0.00]
            poses_from_memory = np.vstack( (C.getFrame(last_box_memory[-2]).getPosition(), C.getFrame(last_box_memory[-1]).getPosition()) )
            middle_between_towers = np.mean(poses_from_memory, 0) + offset_xyz

            dir_vec = C.getFrame(last_box_memory[-2]).getPosition() - C.getFrame(last_box_memory[-1]).getPosition()
            pos2 = C.getFrame(last_box_memory[-2]).getPosition()
            pos1 = C.getFrame(last_box_memory[-1]).getPosition()
            rel_pos =  (pos1 + pos2) / 2 + np.array([0, 0, 0.08])
            
            dir_vec /= np.linalg.norm(dir_vec)

            M.komo.addObjective([2.], ry.FS.position, [box], ry.OT.eq, [1e3], rel_pos)
            M.komo.addObjective([2.], ry.FS.vectorZ, [box], ry.OT.eq, [1e1], dir_vec)
            M.komo.addObjective([2.], ry.FS.vectorZ, [gripper], ry.OT.eq, [1e1], [0, 0, 1])


            
        if l == 1:
            M.place_box(2., box, table, palm, placeDirection)

            M.komo.addObjective([2.], ry.FS.negDistance, [box, last_box_memory[-1]], ry.OT.eq, [1e1], [-.04])
            M.komo.addObjective([2.], ry.FS.scalarProductXY, [box, last_box_memory[-1]], ry.OT.eq, [1e1], [0])
            M.komo.addObjective([2.], ry.FS.positionRel, [box, last_box_memory[-1]], ry.OT.eq, [1.], [.0, .0, .1])
            M.auto_pairwise_keep_dist_([], keep_dist_between, margin = 0.08)

        M.no_collision([], palm, table)
        # keep distance between boxes when placed defined by margin
        # x, y, z, smooth - table size: array([2.5 , 2.5 , 0.1 , 0.02])
        # place inital block as chosen by np.random.choice at a random location
        if l == 0:
            M.place_box(2., box, table, palm, placeDirection)

            M.target_relative_xy_position(2., box, table, [np.random.uniform(0.0, 0.3), np.random.uniform(0.0, 0.3)])
            M.auto_pairwise_keep_dist_([], keep_dist_between, margin = 0.08)


        ways = M.solve()

        if not M.feasible:
            continue

        M2 = M.sub_motion(0)
        # M = manip.ManipulationModelling(C, info, helpers=[gripper])
        # M.setup_point_to_point_motion(qStart, ways[0])
        M2.no_collision([.3,.7], palm, box)
        
        M2.retract([.0, .2], gripper, dist = .13)
        M2.approach([.8, 1.], gripper)
        M2.solve()
        if not M2.feasible:
            continue

        M3 = M.sub_motion(1)
        #manip.ManipulationModelling(C, info)
        # M.setup_point_to_point_motion(ways[0], ways[1])
        M3.no_collision([], table, box)

        M3.solve()
        if not M3.ret.feasible:
            continue

        M2.play(C)
        C.attach(gripper, box)
        M3.play(C)
        C.attach(table, box)
        # update memory
        last_box_memory.append(box)
        # remove box that was previously used
        boxes.remove(box)
        # update positions
        boxes_pos = [C.frame(box1).getPosition(), C.frame(box2).getPosition(), C.frame(box3).getPosition()]


del M

=== placement 0: grasp x place z
place direction: z
  -- feasible:placement 0: grasp x place z
     { time: 0.013718, evals: 30, done: 1, feasible: 1, sos: 0.0838231, f: 0, ineq: 3.26369e-06, eq: 0.0273005 }
  -- feasible:sub_motion_0--placement 0: grasp x place z
     { time: 0.056498, evals: 33, done: 1, feasible: 1, sos: 2.89379, f: 0, ineq: 0, eq: 0.475814 }
  -- feasible:sub_motion_1--placement 0: grasp x place z
     { time: 0.068866, evals: 39, done: 1, feasible: 1, sos: 0.318505, f: 0, ineq: 0.000509541, eq: 0.323601 }
=== placement 1: grasp x place z
place direction: z
  -- feasible:placement 1: grasp x place z
     { time: 0.045109, evals: 66, done: 1, feasible: 1, sos: 0.0965918, f: 0, ineq: 0.205307, eq: 0.455776 }
  -- feasible:sub_motion_0--placement 1: grasp x place z
     { time: 0.029183, evals: 19, done: 1, feasible: 1, sos: 2.051, f: 0, ineq: 6.61768e-07, eq: 0.318815 }
  -- feasible:sub_motion_1--placement 1: grasp x place z
     { time: 0.355785, evals: 201, done: 